# Constructive heuristics comparison

The objective of the $\alpha$-neighbor $p$-center problem can be thought of as "equally" distributing the facilities among the clients to cover them efficiently.

This is the actual goal of the $p$-dispersion problem, so a constructive heuristic that uses its objective function will be tested and compared against a greedy heuristic that takes into account the objective function of this problem.

There will be used 20 random instances of size $n = 50$, $p = 5$ and other 20 of size $n = 400$, $p = 20$, and each one will be tested with both $\alpha = 2$ and $\alpha = 3$. The coordinates of the points are between 0 and 1000 for both planes.

In [1]:
from copy import deepcopy
from typing import List

from models.instance import Instance


def generate_instances(amount: int, n: int, p: int) -> List[Instance]:
    alpha2 = [
        Instance.random(n, p, 2, 1000, 1000)
        for _ in range(amount)
    ]
    alpha3 = deepcopy(alpha2)
    for i in alpha3:
        i.alpha = 3
    return alpha2 + alpha3

In [2]:
instances = generate_instances(20, 50, 5) + generate_instances(20, 400, 20)

We will use the following code to measure the time taken by the evaluations and the objective function results, formatted in a Pandas DataFrame.

In [3]:
import timeit

import pandas as pd

from heuristics.constructive import pdp_based, greedy
from utils import eval_obj_func

def measure(instance, heuristic):
    start = timeit.default_timer()
    solution = heuristic(instance)
    time = timeit.default_timer() - start
    of = eval_obj_func(instance, solution)
    return heuristic.__name__, solution, of, time

def get_dataframe(data):
    return pd.DataFrame({
        colname: [d[i] for d in data]
        for colname, i in zip(
            ('n', 'p', 'a', 'heuristic', 'solution', 'OF', 'seconds'),
            range(len(data[0])))
    })

## Comparing data

Let's create the dataframe of PDP-based evaluations, or get it from a file if it already exists:

In [4]:
import os

OUT_FOLDER = 'nb_results\\constructive'

filepath = os.path.join(OUT_FOLDER, 'pdp_df.csv')
if os.path.exists(filepath):
    pdp_df = pd.read_csv(filepath)
else:
    pdp_data = [(*i.get_parameters(), *measure(i, pdp_based))  for i in instances]
    pdp_df = get_dataframe(pdp_data)
    pdp_df.to_csv(filepath, index=False)
pdp_df

,n,p,a,heuristic,solution,OF,seconds
0,50,5,2,pdp_based,"{33, 5, 37, 40, 28}",628,0.000760
1,50,5,2,pdp_based,"{33, 2, 34, 13, 14}",607,0.000535
2,50,5,2,pdp_based,"{35, 5, 8, 9, 42}",794,0.000527
3,50,5,2,pdp_based,"{18, 19, 13, 45, 14}",611,0.000617
4,50,5,2,pdp_based,"{1, 7, 27, 29, 47}",643,0.000525
...,...,...,...,...,...,...,...
75,400,20,3,pdp_based,"{66, 68, 198, 74, 395, 397, 399, 16, 81, 209, ...",406,0.074006
76,400,20,3,pdp_based,"{261, 8, 201, 202, 13, 22, 281, 155, 222, 289,...",390,0.093924
77,400,20,3,pdp_based,"{1, 259, 325, 198, 134, 201, 204, 77, 78, 208,...",375,0.073400
78,400,20,3,pdp_based,"{391, 10, 13, 77, 271, 16, 17, 274, 144, 84, 2...",372,0.064396


Now the dataframe of the greedy results:

In [5]:
filepath = os.path.join(OUT_FOLDER, 'greedy_df.csv')
if os.path.exists(filepath):
    greedy_df = pd.read_csv(filepath)
else:
    greedy_data = [(*i.get_parameters(), *measure(i, greedy))  for i in instances]
    greedy_df = get_dataframe(greedy_data)
    greedy_df.to_csv(filepath, index=False)
greedy_df

,n,p,a,heuristic,solution,OF,seconds
0,50,5,2,greedy,"{2, 3, 37, 40, 28}",600,0.216955
1,50,5,2,greedy,"{33, 1, 13, 14, 15}",607,0.067130
2,50,5,2,greedy,"{1, 35, 8, 44, 31}",627,0.021097
3,50,5,2,greedy,"{18, 3, 4, 43, 13}",591,0.050969
4,50,5,2,greedy,"{20, 38, 7, 9, 29}",610,0.020568
...,...,...,...,...,...,...,...
75,400,20,3,greedy,"{1, 2, 3, 4, 5, 198, 6, 8, 7, 9, 10, 14, 16, 2...",369,32.796715
76,400,20,3,greedy,"{1, 2, 3, 4, 5, 6, 7, 8, 201, 10, 12, 14, 16, ...",401,36.384875
77,400,20,3,greedy,"{1, 2, 3, 67, 4, 6, 201, 9, 11, 12, 77, 78, 14...",434,33.087493
78,400,20,3,greedy,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 138, 10, 12, 13, 1...",404,29.960488


We now have 2 dataframes, one for each heuristic. Let's filter them by $n$ and $\alpha$ too:

In [15]:
filtered_data = {
    heuristic: {
        f'n{n}': {
            f'a{alpha}': df[
                (df['n'] == n) &
                (df['a'] == alpha)
                ].iloc[:, [0, 1, 2, 3, 5, 6]]
            for alpha in (2, 3)
        }
        for n in (50, 400)
    }
    for heuristic, df in (('pdp', pdp_df), ('greedy', greedy_df))
}

Now we can access the data by using keys referring to the heuristic, its size $n$ and $\alpha$:

In [16]:
filtered_data['pdp']['n50']['a2']

,n,p,a,heuristic,OF,seconds
0,50,5,2,pdp_based,628,0.000760
1,50,5,2,pdp_based,607,0.000535
2,50,5,2,pdp_based,794,0.000527
3,50,5,2,pdp_based,611,0.000617
4,50,5,2,pdp_based,643,0.000525
5,50,5,2,pdp_based,688,0.000509
6,50,5,2,pdp_based,608,0.000518
7,50,5,2,pdp_based,743,0.000526
8,50,5,2,pdp_based,779,0.000502
9,50,5,2,pdp_based,627,0.000532


To calculate some basic statistics about the data, let's create a function that will take parameters $n$ and $\alpha$ to compare the results between the 2 heuristics:

In [7]:
def calc_stats(n, a):
    ncol = f'n{n}'
    acol = f'a{a}'
    stats = (filtered_data['pdp'][ncol][acol]
        .compare(filtered_data['greedy'][ncol][acol], keep_equal=True)
        .rename(columns={ 'self': 'pdp', 'other': 'greedy' })
        .drop(columns='heuristic'))

    stats['OF', 'absolute'] = stats['OF', 'pdp'] - stats['OF', 'greedy']
    stats['OF', '%'] = (stats['OF', 'absolute'] / stats['OF', 'pdp']) * 100

    order = ['pdp', 'greedy', 'absolute', '%']
    tops = ('OF', 'seconds')
    stats = stats.loc[:, (tops, order)]

    winnings = [
        stats[stats['OF', 'absolute'] <= 0].count()[0],
        stats[stats['OF', 'absolute'] > 0].count()[0],
        '', '', '', ''
    ]
    average = [
        stats[col].mean()
        for col in stats.columns
    ]

    stats.loc['winnings'] = winnings
    stats.loc['average'] = average

    return stats

In [17]:
from itertools import product

from IPython.display import display

for n, a in product((50, 400), (2, 3)):
    print('------------------------')
    print(f'n = {n}, alpha = {a}')
    stats = calc_stats(n, a)
    filepath = os.path.join(OUT_FOLDER, f'stats_n{n}_a{a}.csv')
    stats.to_csv(filepath)
    display(stats)

------------------------
n = 50, alpha = 2


d:\NETO\programacion\alpha-neighbor-p-center-problem\venv\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


OF                               seconds          
            pdp  greedy absolute          %       pdp    greedy
0         628.0  600.00       28   4.458599   0.00076  0.216955
1         607.0  607.00        0        0.0  0.000535   0.06713
2         794.0  627.00      167  21.032746  0.000527  0.021097
3         611.0  591.00       20   3.273322  0.000617  0.050969
4         643.0  610.00       33   5.132193  0.000525  0.020568
5         688.0  548.00      140  20.348837  0.000509  0.032279
6         608.0  577.00       31   5.098684  0.000518  0.016759
7         743.0  558.00      185  24.899058  0.000526  0.019974
8         779.0  582.00      197  25.288832  0.000502  0.017632
9         627.0  579.00       48   7.655502  0.000532  0.020344
10        881.0  573.00      308  34.960272  0.001564  0.019064
11        669.0  553.00      116  17.339312  0.000799  0.014109
12        724.0  659.00       65   8.977901  0.001302  0.018006
13        765.0  575.00      190  24.836601  0.003575  0.021995
14        663.0  554.00      109  16.440422  0.001014  0.016742
15        861.0  619.00      242  28.106852  0.000935  0.024842
16        767.0  633.00      134  17.470665  0.000962  0.020961
17        747.0  693.00       54   7.228916  0.000582  0.020084
18        638.0  490.00      148  23.197492  0.000528  0.016043
19        565.0  559.00        6   1.061947  0.000601  0.016754
winnings    1.0   19.00                                        
average   700.4  589.35   111.05  14.840408  0.000871  0.033615

------------------------
n = 50, alpha = 3


d:\NETO\programacion\alpha-neighbor-p-center-problem\venv\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


...

d:\NETO\programacion\alpha-neighbor-p-center-problem\venv\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


OF                              seconds           
            pdp greedy absolute          %       pdp     greedy
40        307.0  304.0        3   0.977199   0.07164   45.30798
41        292.0  350.0      -58 -19.863014  0.102789  42.235033
42        310.0  309.0        1   0.322581   0.12521  50.214393
43        294.0  426.0     -132 -44.897959  0.051584  45.909697
44        317.0  297.0       20   6.309148  0.057793  48.009938
45        297.0  358.0      -61 -20.538721  0.049126  51.167755
46        328.0  336.0       -8  -2.439024   0.05111  45.846795
47        323.0  354.0      -31  -9.597523  0.050484  45.359293
48        275.0  362.0      -87 -31.636364  0.052707  49.758392
49        339.0  341.0       -2  -0.589971  0.062176  50.711559
50        304.0  277.0       27   8.881579   0.05425  48.212123
51        324.0  362.0      -38 -11.728395   0.05771  51.083328
52        294.0  315.0      -21  -7.142857  0.053468  50.279512
53        312.0  336.0      -24  -7.692308  0.056241   58.75892
54        276.0  308.0      -32 -11.594203   0.05742  55.484057
55        348.0  302.0       46  13.218391  0.053719  52.859262
56        359.0  299.0       60  16.713092  0.057072  44.194823
57        343.0  348.0       -5  -1.457726  0.056023  50.252007
58        276.0  348.0      -72 -26.086957  0.058349  38.374765
59        284.0  270.0       14   4.929577  0.058375  43.111878
winnings   13.0    7.0                                         
average   310.1  330.1    -20.0  -7.195673  0.061862  48.356575

------------------------
n = 400, alpha = 3


d:\NETO\programacion\alpha-neighbor-p-center-problem\venv\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


OF                               seconds           
             pdp  greedy absolute          %       pdp     greedy
60        409.00  425.00      -16   -3.91198   0.06933   30.42869
61        353.00  480.00     -127 -35.977337  0.056991  29.794342
62        400.00  389.00       11       2.75  0.057283  34.427897
63        386.00  431.00      -45 -11.658031  0.055634  30.758092
64        350.00  395.00      -45 -12.857143  0.059793  32.295813
65        377.00  511.00     -134 -35.543767  0.053938  33.588127
66        357.00  407.00      -50 -14.005602  0.057638  30.638826
67        422.00  361.00       61  14.454976  0.053892  34.784959
68        372.00  444.00      -72 -19.354839  0.059268  99.633402
69        471.00  414.00       57  12.101911  0.058515  64.141011
70        365.00  419.00      -54 -14.794521  0.052566  43.250826
71        377.00  383.00       -6  -1.591512  0.054641  42.673987
72        347.00  366.00      -19  -5.475504   0.05553   35.28277
73        397.00  421.00      -24   -6.04534  0.075509  33.756992
74        354.00  391.00      -37 -10.451977  0.093988  34.871092
75        406.00  369.00       37     9.1133  0.074006  32.796715
76        390.00  401.00      -11  -2.820513  0.093924  36.384875
77        375.00  434.00      -59 -15.733333    0.0734  33.087493
78        372.00  404.00      -32  -8.602151  0.064396  29.960488
79        341.00  366.00      -25  -7.331378  0.097384  37.203826
winnings   16.00    4.00                                         
average   381.05  410.55    -29.5  -8.386737  0.065881  38.988011

The greedy heuristic performs better with small instances, with a minimal difference in time taken compared to the PDP-based, both of them taking less than 1 second.
However, with bigger instances the greedy takes more than 30 seconds while also performing worse.
The PDP-based takes more time solving big instances but it's still less than a second.

As a result, we can conclude that the greedy heuristic is only worth using in small instances.